In [1]:
import pymysql
pymysql.install_as_MySQLdb()
import copy
import numpy as np
import pandas as pd
import logging
import requests
from bs4 import BeautifulSoup
import random

from adobe_analytics import Client, ReportDefinition
from datetime import datetime, timedelta

In [2]:
def create_db_instance():
    db = pymysql.connect(
        host="writter-db.chsokvgtm8us.us-east-2.rds.amazonaws.com",
        port=3306,
        user="root",
        passwd="12345678",
        db="writter"
    )
    
    return db
    
def close_db_instance(db):
    db.close()

In [3]:
db = create_db_instance()

In [4]:
def create_fake_user(db, user):
    cur = db.cursor()
    
    sql = 'INSERT INTO `writter`.`users`'
    sql += '('
    sql += '`email`,'
    sql += '`first_name`,'
    sql += '`last_name`,'
    sql += '`password`,'
    sql += '`points`,'
    sql += '`enabled`,'
    sql += '`created`,'
    sql += '`modified`,'
    sql += '`username`)'
    sql += 'VALUES'
    sql += '('
    sql += '"' + user['email'] + '",'
    sql += '"' + user['first_name'] + '",'
    sql += '"' + user['last_name'] + '",'
    sql += '"' + '$2a$10$9L8exgqgB.yvoN0km3d.5uRjpfscFMjwADUmGizMW58TBZS8wsj5S' + '",'
    sql += '0' + ','
    sql += '1' + ','
    sql += '"' + datetime.now().isoformat() + '",'
    sql += '"' + datetime.now().isoformat() + '",'
    sql += '"' + user['username'] + '");'
    
    cur.execute(sql)
    db.commit()
    cur.close()

    return user

In [5]:
names = []
with open("names.txt", "r") as ins:
    for line in ins:
        line = line.strip().split(' ')
        names.append({
            'first_name': line[0],
            'last_name': line[1],
        })

In [6]:
for name in names:
    user = {}
    user['first_name'] = name['first_name']
    user['last_name'] = name['last_name']
    user['email'] = user['first_name'][0].lower() + user['last_name'].lower() + '@fakeuser.com'
    user['username'] = user['first_name'][0].lower() + user['last_name'].lower()
    try:
        create_fake_user(db, user)
    except Exception as e:
        print('error', e)
        
    

error (1062, "Duplicate entry 'asparks@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'ofarley@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'xbanks@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'pcobb@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'ahill@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'dfrancis@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'jprice@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'thaney@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'zsanders@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'ijackson@fakeuser.com' for key 'UK_6dotkott2kjsp8vw4d0m25fb7'")
error (1062, "Duplicate entry 'tduarte@fakeuser.com' for key 'UK

In [7]:
def get_fake_users(db):
    cur = db.cursor()

    sql = 'SELECT id, email FROM '
    sql += 'writter.users '
    sql += "WHERE email LIKE '%fakeuser.com%'"
    
    cur.execute(sql)
    
    users = []
    
    for row in cur.fetchall():
        user = {
            'id': row[0],
            'email': row[1],
        }
        users.append(user)
    cur.close()
    return users

In [8]:
fake_users = get_fake_users(db)

In [9]:
def create_stories(db, story):
    user_index = random.randint(1, (len(fake_users) - 1))
    cur = db.cursor()
    
    sql = "INSERT INTO `writter`.`stories` (`author_id`, `created`, `enabled`, `modified`, `points`, `text`, `title`, `views`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    vals = (fake_users[user_index]['id'], datetime.now().isoformat(), 1, datetime.now().isoformat(), 0, story['text'], story['name'], 0)
    cur.execute(sql, vals)
    
    db.commit()
    cur.close()

    return user

In [10]:
url = 'http://textfiles.com/stories/'
page = requests.get(url)

In [11]:
soup = BeautifulSoup(page.content, 'html.parser')

In [12]:
trs = soup.find_all('tr')

stories = []
for tr in trs[3].text.splitlines():
    file_name = tr.split(' ')[0]
    story_name = ' '.join(tr.split(' ')[3:])
    story_name = story_name.split('(')[0]
    story_name = story_name.split(',')[0]
    story_name = story_name.split('by')[0]
    story_name = story_name.strip()
    stories.append({'file': file_name, 'name': story_name})
#     print(file_name, '|', story_name)
print(len(stories))

452


In [13]:
for story in stories:
    full_url = url + story['file']
    page = requests.get(full_url)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    text = soup.text.strip()
    story['text'] = text
    
    try:
        create_stories(db, story)
    except Exception as e:
        print('error', e)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
C:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xEF\\xBF\\xBDBU\\x17...' for column 'text' at row 1")
  result = self._query(query)
C:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE0\\xB8\\xA9\\xE0\\xB8\\xAD...' for column 'text' at row 1")
  result = self._query(query)
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
C:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xEF\\xBF\\xBD\\xC3\\xA8\\xEF...' for column 'text' at row 1")
  result = self._query(query)


In [14]:
def get_stories(db):
    cur = db.cursor()

    sql = 'SELECT id, title FROM '
    sql += 'writter.stories'
    
    cur.execute(sql)
    
    stories = []
    
    for row in cur.fetchall():
        story = {
            'id': row[0],
            'title': row[1],
        }
        stories.append(story)
    cur.close()
    return stories

In [15]:
all_stories = get_stories(db)

In [16]:
# def create_authors(db, story, user):
#     cur = db.cursor()
    
#     sql = "INSERT INTO `writter`.`authors` (`story_id`, `user_id`) VALUES (%s, %s)"
#     vals = (story, user)
#     cur.execute(sql, vals)
    
#     db.commit()
#     cur.close()

#     return user

In [17]:
# for story in all_stories:
#     user_index = random.randint(1, (len(fake_users) - 1))
#     create_authors(db, story['id'], fake_users[user_index]['id'])

In [18]:
def create_rating(db, story, user):
    rating_range = random.randint(1, 100)
    rating = 4
    if rating_range <= 10:
        rating = 1
    elif rating_range <= 25:
        rating = 2
    elif rating_range <= 50:
        rating = 3
    elif rating_range <= 80:
        rating = 4
    else:
        rating = 5

    cur = db.cursor()
    
    sql = "INSERT INTO `writter`.`ratings` (`created`, `enabled`, `modified`, `rating`, `story_id`, `user_id`) VALUES (%s, %s, %s, %s, %s, %s)"
    vals = (datetime.now().isoformat(), 1, datetime.now().isoformat(), rating, story, user)
    cur.execute(sql, vals)
    
    db.commit()
    cur.close()

    return user

In [19]:
for story in all_stories:
    number_of_users_rating = random.randint(5, 20)
    for _ in range(0, number_of_users_rating):
        user_index = random.randint(1, (len(fake_users) - 1))
        create_rating(db, story['id'], fake_users[user_index]['id'])

In [20]:
close_db_instance(db)